In [1]:
import pandas as pd
import numpy as np

# 1. Load data


In [25]:
data_path = "/Users/francoismoreau/Documents/Polytechnique/4A/Data Challenge Health/ChallengeXHEC23022024.xlsx"
dict_df_sheets = pd.read_excel(data_path, sheet_name=None)
df_hist_jan = dict_df_sheets["JAN24"].copy()
df_clients = dict_df_sheets["clients"].copy()
df_inter = dict_df_sheets["intervenants"].copy()

In [26]:
df_hist_jan.head()

,ID Client,ID Intervenant,Date,Heure de début,Heure de fin,Prestation
0,559475456,162858075,2024-01-01,07:15:00,07:45:00,REPAS
1,559277088,162858075,2024-01-01,07:45:00,08:30:00,TOILETTE
2,87852633,78007018,2024-01-01,07:45:00,08:30:00,TOILETTE
3,243033408,810259688,2024-01-01,07:45:00,08:15:00,TOILETTE
4,814940942,710283561,2024-01-01,07:45:00,09:20:00,TOILETTE


In [27]:
df_clients.head()

,ID Client,Latitude,Longitude
0,559475456,48.721052,1.375689
1,559277088,48.721052,1.375689
2,87852633,48.691944,1.498773
3,243033408,48.726393,1.332848
4,814940942,48.733174,1.370689


In [28]:
df_inter.head()

,ID Intervenant,Latitude,Longitude,Compétences,Permis,Véhicule personnel,Dispo / Indispo
0,838320706,48.738516,1.391971,"AIDE MENAGERE, REPAS, TOILETTE",Oui,Oui,"Indispo 01/01, 30/01, 31/01"
1,609468992,48.640555,1.232581,"TOILETTE, REPAS, VIE SOCIALE, AIDE MENAGERE",Oui,Oui,"Indispo Tous les mercredis + 13/01, 14/01, 27/..."
2,78012267,48.729206,1.371985,"HOMMES TOUTES MAINS, JARDINAGE",Oui,Oui,Dispo le 25/01
3,818696864,48.744702,1.357921,"REPAS, AIDE MENAGERE, ACCOMPAGNEMENTS COURSES,...",Oui,Oui,Indispo tous les samedis et dimanche
4,746414886,48.769455,1.197644,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,Indispo tous les mercredis + le 05/01


# 2. Data exploration preprocessing


In [29]:
df_hist_jan.rename(
    {
        "ID Client": "id_client",
        "ID Intervenant": "id_inter",
        "Date": "date",
        "Heure de début": "start",
        "Heure de fin": "end",
        "Prestation": "obj",
    },
    axis=1,
    inplace=True,
)

df_clients.rename(
    {"ID Client": "id_client", "Latitude": "lat_client", "Longitude": "lon_client"},
    axis=1,
    inplace=True,
)

df_inter.rename(
    {
        "ID Intervenant": "id_inter",
        "Latitude": "lat_inter",
        "Longitude": "lon_inter",
        "Compétences": "skills",
        "Permis": "license",
        "Véhicule personnel": "car",
        "Dispo / Indispo": "free",
    },
    axis=1,
    inplace=True,
)

In [30]:
df_hist_jan["date"] = pd.to_datetime(df_hist_jan["date"])
df_hist_jan["start"] = pd.to_datetime(df_hist_jan["date"])
df_hist_jan["date"] = pd.to_datetime(df_hist_jan["date"])

In [31]:
df_hist_jan

,id_client,id_inter,date,start,end,obj
0,559475456,162858075,2024-01-01,2024-01-01,07:45:00,REPAS
1,559277088,162858075,2024-01-01,2024-01-01,08:30:00,TOILETTE
2,87852633,78007018,2024-01-01,2024-01-01,08:30:00,TOILETTE
3,243033408,810259688,2024-01-01,2024-01-01,08:15:00,TOILETTE
4,814940942,710283561,2024-01-01,2024-01-01,09:20:00,TOILETTE
...,...,...,...,...,...,...
2865,559277088,710283561,2024-01-31,2024-01-31,19:15:00,REPAS
2866,714782168,810259688,2024-01-31,2024-01-31,20:00:00,REPAS
2867,559475456,710283561,2024-01-31,2024-01-31,20:00:00,TOILETTE
2868,803656603,854577575,2024-01-31,2024-01-31,19:50:00,TOILETTE


# 3. EDA


In [32]:
nb_clients_jan = len(df_hist_jan["id_client"].unique())
print(f"Number of clients in January: {nb_clients_jan}")
nb_inter_jan = len(df_hist_jan["id_inter"].unique())
print(f"Number of intervenants in January: {nb_inter_jan}")

Number of clients in January: 118
Number of intervenants in January: 24


# 4. Modeling with PuLP


See precedent version


# 5. Modeling with Google OR Tools


## 5.A. Documentation


See precedent version


## 5.B. Tests with synthetic data


### 5.B.1. With initial locations and time windows


In [3]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from functools import partial

In [43]:
def create_simple_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix"] = [
        [0, 3, 6, 8, 9, 5, 6],
        [3, 0, 3, 5, 7, 4, 5],
        [6, 3, 0, 3, 5, 4, 6],
        [8, 5, 3, 0, 3, 8, 4],
        [9, 7, 5, 3, 0, 9, 3],
        [5, 4, 4, 8, 9, 0, 8],
        [6, 5, 6, 4, 3, 8, 0],
    ]
    data["time_windows"] = [
        (0, 15),  # 1
        (15, 30),  # 2
        (30, 45),  # 3
        (30, 45),  # 4
        (0, 15),  # 5
        (0, 200),  # 6
        (0, 200),  # 7
    ]
    data["num_vehicles"] = 2
    data["starts"] = [5, 6]
    data["ends"] = [5, 6]
    return data

In [44]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)+1}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)+1}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")

In [45]:
def main():
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_simple_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["time_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Time Windows constraint.
    time = "Time"
    routing.AddDimension(
        transit_callback_index,
        30,  # allow waiting time
        150,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time,
    )
    time_dimension = routing.GetDimensionOrDie(time)
    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    # # Add time window constraints for each vehicle start node.
    # depot_idx = data["depot"]
    # for vehicle_id in range(data["num_vehicles"]):
    #     index = routing.Start(vehicle_id)
    #     time_dimension.CumulVar(index).SetRange(
    #         data["time_windows"][depot_idx][0], data["time_windows"][depot_idx][1]
    #     )

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [46]:
main()

Objective: 25
Route for vehicle 0:
6 Time(0,0) -> 1 Time(5,15) -> 2 Time(15,27) -> 3 Time(30,30) -> 6 Time(34,34)
Time of the route: 34min

Route for vehicle 1:
7 Time(0,0) -> 5 Time(3,15) -> 4 Time(30,30) -> 7 Time(34,34)
Time of the route: 34min

Total time of all routes: 68min


### 5.B.2. With initial locations, time windows, and bike/car time travel


In [48]:
def create_simple_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix_car"] = [
        [0, 3, 6, 8, 9, 5, 6],
        [3, 0, 3, 5, 7, 4, 5],
        [6, 3, 0, 3, 5, 4, 6],
        [8, 5, 3, 0, 3, 8, 4],
        [9, 7, 5, 3, 0, 9, 3],
        [5, 4, 4, 8, 9, 0, 8],
        [6, 5, 6, 4, 3, 8, 0],
    ]
    data["time_matrix_bike"] = [
        [0, 5, 8, 10, 11, 7, 8],
        [5, 0, 5, 7, 9, 6, 7],
        [8, 5, 0, 5, 7, 6, 8],
        [10, 7, 5, 0, 5, 10, 6],
        [11, 9, 7, 5, 0, 11, 5],
        [7, 6, 6, 10, 11, 0, 10],
        [8, 7, 8, 6, 5, 10, 0],
    ]
    data["time_windows"] = [
        (0, 15),  # 1
        (15, 30),  # 2
        (30, 31),  # 3
        (30, 31),  # 4
        (0, 15),  # 5
        (0, 200),  # 6
        (0, 200),  # 7
    ]
    data["num_vehicles"] = 2
    data["is_car"] = [0, 1]
    data["starts"] = [5, 6]
    data["ends"] = [5, 6]
    return data

In [49]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        if data["is_car"][vehicle_id] == 1:
            type_str = "car"
        else:
            type_str = "bike"
        plan_output = f"Route for vehicle {vehicle_id} ({type_str}):\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)+1}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)+1}"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")

In [50]:
def main():
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_simple_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node]
        else:
            return data["time_matrix_bike"][from_node][to_node]

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)
    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [bike_index, car_index],
        100,  # no slack
        [300, 300],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)
    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)

In [51]:
main()

Objective: 33
Route for vehicle 0 (bike):
6 Time(0,0) -> 1 Time(7,15) -> 2 Time(15,25) -> 3 Time(30,30) -> 6 Time(36,36)
Time of the route: 36min

Route for vehicle 1 (car):
7 Time(0,0) -> 5 Time(3,15) -> 4 Time(30,30) -> 7 Time(34,34)
Time of the route: 34min

Total time of all routes: 70min


### 5.B.3. With initial locations, time windows, bike/car time travel and task execution time


In [52]:
def create_simple_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix_car"] = [
        [0, 3, 6, 8, 9, 5, 6],
        [3, 0, 3, 5, 7, 4, 5],
        [6, 3, 0, 3, 5, 4, 6],
        [8, 5, 3, 0, 3, 8, 4],
        [9, 7, 5, 3, 0, 9, 3],
        [5, 4, 4, 8, 9, 0, 8],
        [6, 5, 6, 4, 3, 8, 0],
    ]
    data["time_matrix_bike"] = [
        [0, 5, 8, 10, 11, 7, 8],
        [5, 0, 5, 7, 9, 6, 7],
        [8, 5, 0, 5, 7, 6, 8],
        [10, 7, 5, 0, 5, 10, 6],
        [11, 9, 7, 5, 0, 11, 5],
        [7, 6, 6, 10, 11, 0, 10],
        [8, 7, 8, 6, 5, 10, 0],
    ]
    data["service_times"] = [1, 2, 3, 4, 5, 0, 0]
    data["time_windows"] = [
        (0, 15),  # 1
        (15, 30),  # 2
        (30, 45),  # 3
        (30, 45),  # 4
        (0, 15),  # 5
        (0, 200),  # 6
        (0, 200),  # 7
    ]
    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1],
        )
        for i in range(len(data["service_times"]))
    ]  # Requires that time windows let the time to execute the service
    data["num_vehicles"] = 2
    data["is_car"] = [0, 1]
    data["starts"] = [5, 6]
    data["ends"] = [5, 6]
    return data

In [53]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        if data["is_car"][vehicle_id] == 1:
            type_str = "car"
        else:
            type_str = "bike"
        plan_output = f"Route for vehicle {vehicle_id} ({type_str}):\n"
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += (
                f"{manager.IndexToNode(index)+1}"
                f" Time({solution.Min(time_var)},{solution.Max(time_var)})"
                " -> "
            )
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += (
            f"{manager.IndexToNode(index)+1}"
            # f"(Slack var: {time_dimension.SlackVar(index)})"
            f" Time({solution.Min(time_var)},{solution.Max(time_var)})\n"
        )
        plan_output += f"Time of the route: {solution.Min(time_var)}min\n"
        print(plan_output)
        total_time += solution.Min(time_var)
    print(f"Total time of all routes: {total_time}min")

In [54]:
def print_detailed_solution(data, manager, routing, solution):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        print(f"Route for vehicle {vehicle_id}:")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            service_time = data["service_times"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                if data["is_car"][vehicle_id]:
                    next_travel_time = data["time_matrix_car"][
                        manager.IndexToNode(index)
                    ][next_node]
                else:
                    next_travel_time = data["time_matrix_bike"][
                        manager.IndexToNode(index)
                    ][next_node]
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )

            break_time = departure_time - end_of_service_time

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            # Print detailed timing for current location
            print(
                f"Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time}, "
                f"End of Service Time: {end_of_service_time}, Break: {break_time}, "
                f"Departure: {departure_time}) -> "
            )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        print(
            f"Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time})"
        )
        total_time += final_arrival_time
        print("\n")
    print(f"Total time of all routes: {total_time}min")

In [55]:
def main():
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_simple_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node] + service_time
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)
    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [bike_index, car_index],
        100,  # no slack
        [300, 300],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        print_detailed_solution(data, manager, routing, solution)

In [56]:
main()

Objective: 48
Route for vehicle 0:
Location 6 (Arrival: 0, End of Service Time: 0, Break: 0, Departure: 0) -> 
Location 1 (Arrival: 7, End of Service Time: 8, Break: 2, Departure: 10) -> 
Location 2 (Arrival: 15, End of Service Time: 17, Break: 0, Departure: 17) -> 
Location 6 (Arrival: 23)


Route for vehicle 1:
Location 7 (Arrival: 0, End of Service Time: 0, Break: 0, Departure: 0) -> 
Location 5 (Arrival: 3, End of Service Time: 8, Break: 19, Departure: 27) -> 
Location 4 (Arrival: 30, End of Service Time: 34, Break: 0, Departure: 34) -> 
Location 3 (Arrival: 37, End of Service Time: 40, Break: 0, Departure: 40) -> 
Location 7 (Arrival: 46)


Total time of all routes: 69min


### 5.B.4. With initial locations, time windows, bike/car time travel, task execution time and skills


In [74]:
def create_simple_data_model():
    """Stores the data for the problem."""
    data = {}
    data["time_matrix_car"] = [
        [0, 3, 6, 8, 9, 5, 6],
        [3, 0, 3, 5, 7, 4, 5],
        [6, 3, 0, 3, 5, 4, 6],
        [8, 5, 3, 0, 3, 8, 4],
        [9, 7, 5, 3, 0, 9, 3],
        [5, 4, 4, 8, 9, 0, 8],
        [6, 5, 6, 4, 3, 8, 0],
    ]
    data["time_matrix_bike"] = [
        [0, 5, 8, 10, 11, 7, 8],
        [5, 0, 5, 7, 9, 6, 7],
        [8, 5, 0, 5, 7, 6, 8],
        [10, 7, 5, 0, 5, 10, 6],
        [11, 9, 7, 5, 0, 11, 5],
        [7, 6, 6, 10, 11, 0, 10],
        [8, 7, 8, 6, 5, 10, 0],
    ]
    data["service_times"] = [1, 2, 3, 4, 5, 0, 0]
    data["time_windows"] = [
        (0, 15),  # 1
        (15, 30),  # 2
        (30, 45),  # 3
        (30, 45),  # 4
        (0, 15),  # 5
        (0, 200),  # 6
        (0, 200),  # 7
    ]
    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1],
        )
        for i in range(len(data["service_times"]))
    ]  # Requires that time windows let the time to execute the service
    data["num_vehicles"] = 2
    data["is_car"] = [0, 1]
    data["starts"] = [5, 6]
    data["ends"] = [5, 6]

    # Add skills for each vehicle
    data["vehicle_skills"] = [["skill1", "skill2"], ["skill2"]]
    # Add skill requirements for each node
    data["node_requirements"] = [
        None,
        "skill1",
        "skill2",
        "skill1",
        "skill2",
        None,
        None,
    ]
    return data

In [75]:
def print_detailed_solution(data, manager, routing, solution):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        vehicle_skills = str(data["vehicle_skills"][vehicle_id])
        print(f"Route for vehicle {vehicle_id} (Skills: {vehicle_skills}):")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            service_time = data["service_times"][manager.IndexToNode(index)]
            skills_required = data["node_requirements"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                if data["is_car"][vehicle_id]:
                    next_travel_time = data["time_matrix_car"][
                        manager.IndexToNode(index)
                    ][next_node]
                else:
                    next_travel_time = data["time_matrix_bike"][
                        manager.IndexToNode(index)
                    ][next_node]
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )

            break_time = departure_time - end_of_service_time

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            # Print detailed timing for current location
            print(
                f"Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time}, "
                f"End of Service Time (Skills required: {skills_required}): {end_of_service_time}, Break: {break_time}, "
                f"Departure: {departure_time}) -> "
            )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        print(
            f"Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time})"
        )
        total_time += final_arrival_time
        print("\n")
    print(f"Total time of all routes: {total_time}min")

In [76]:
def main():
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_simple_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node] + service_time
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)
    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)

    # Enforce node requirements based on vehicle skills
    for node_idx, requirement in enumerate(data["node_requirements"]):
        if requirement:  # Check if the node has specific skill requirements
            allowed_vehicles = [
                vehicle_id
                for vehicle_id, skills in enumerate(data["vehicle_skills"])
                if requirement in skills
            ]
            manager_idx = manager.NodeToIndex(node_idx)
            if manager_idx >= 0:  # Ensure manager index is valid
                routing.SetAllowedVehiclesForIndex(allowed_vehicles, manager_idx)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [bike_index, car_index],
        100,  # no slack
        [300, 300],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        print_detailed_solution(data, manager, routing, solution)

In [77]:
main()

Objective: 51
Route for vehicle 0 (Skills: ['skill1', 'skill2']):
Location 6 (Arrival: 0, End of Service Time (Skills required: None): 0, Break: 0, Departure: 0) -> 
Location 1 (Arrival: 7, End of Service Time (Skills required: None): 8, Break: 2, Departure: 10) -> 
Location 2 (Arrival: 15, End of Service Time (Skills required: skill1): 17, Break: 6, Departure: 23) -> 
Location 4 (Arrival: 30, End of Service Time (Skills required: skill1): 34, Break: 0, Departure: 34) -> 
Location 3 (Arrival: 39, End of Service Time (Skills required: skill2): 42, Break: 0, Departure: 42) -> 
Location 6 (Arrival: 48)


Route for vehicle 1 (Skills: ['skill2']):
Location 7 (Arrival: 0, End of Service Time (Skills required: None): 0, Break: 0, Departure: 0) -> 
Location 5 (Arrival: 3, End of Service Time (Skills required: skill2): 8, Break: 0, Departure: 8) -> 
Location 7 (Arrival: 11)


Total time of all routes: 59min


### 5.C. With initial locations, time windows, bike/car time travel, task execution time and skills with REAL DATA


In [4]:
c_c_dist = pd.read_csv("client_client_distance.csv")
c_l_need = pd.read_csv("client_loc_need.csv")
c_n_dist = pd.read_csv("client_nurse_distance.csv")

list_clients_1 = c_c_dist["ID Client 1"].unique()
list_clients_2 = c_c_dist["ID Client 2"].unique()
full_client_list = list(set([*list_clients_1, *list_clients_2]))
nb_full_client = len(full_client_list)

In [5]:
client_id_to_idx = {}
client_idx_to_id = {}
for k in range(len(full_client_list)):
    client_id_to_idx[full_client_list[k]] = k
    client_idx_to_id[k] = full_client_list[k]

In [6]:
def convert_duration(str_duration):
    if "hour" in str_duration:
        list = str_duration.split()
        nb_minutes = int(list[0]) * 60 + int(list[2])
        return nb_minutes
    else:
        list = str_duration.split()
        nb_minutes = int(list[0])
        return nb_minutes

In [7]:
def define_client_time_matrix(
    df, vehicle_type, client_id_to_idx=client_id_to_idx, out_value=4000
):
    list_clients_1 = df["ID Client 1"].unique()
    list_clients_2 = df["ID Client 2"].unique()
    full_list = list(set([*list_clients_1, *list_clients_2]))

    matrix = [[0 for i in range(len(full_list))] for k in range(len(full_list))]
    col_type = "Duration_" + vehicle_type
    for client_1_id in full_list:
        client_1_idx = client_id_to_idx[client_1_id]
        for client_2_id in full_list:
            client_2_idx = client_id_to_idx[client_2_id]
            if client_1_id != client_2_id:
                try:
                    travel_time = convert_duration(
                        df.loc[
                            (df["ID Client 1"] == client_1_id)
                            & (df["ID Client 2"] == client_2_id),
                            col_type,
                        ].values[0]
                    )
                    matrix[client_1_idx][client_2_idx] = travel_time
                except IndexError:
                    try:
                        travel_time = convert_duration(
                            df.loc[
                                (df["ID Client 1"] == client_2_id)
                                & (df["ID Client 2"] == client_1_id),
                                col_type,
                            ].values[0]
                        )
                        matrix[client_1_idx][client_2_idx] = travel_time
                    except IndexError:
                        matrix[client_1_idx][client_2_idx] = out_value

    return np.array(matrix)

In [8]:
client_time_matrix_car = define_client_time_matrix(c_c_dist, vehicle_type="Car")
client_time_matrix_bike = define_client_time_matrix(c_c_dist, vehicle_type="Bike")

In [9]:
list_inter_columns = c_n_dist.columns[2:]
full_inter_list = list(set([int(x.split(sep="_")[2]) for x in list_inter_columns]))

inter_id_to_idx = {}
inter_idx_to_id = {}
for k in range(len(full_inter_list)):
    inter_id_to_idx[full_inter_list[k]] = k
    inter_idx_to_id[k] = full_inter_list[k]

In [10]:
def define_inter_time_matrix(
    df,
    full_client_list,
    inter_id_to_idx=inter_id_to_idx,
    client_id_to_idx=client_id_to_idx,
    vehicle_type="Car",
    out_value=4000,
):
    list_inter_columns = df.columns[2:]
    full_inter_list = [int(x.split(sep="_")[2]) for x in list_inter_columns]

    matrix = [
        [0 for i in range(len(full_client_list))] for k in range(len(full_inter_list))
    ]

    for inter_id in full_inter_list:
        vehicle_str = "_Driving" if vehicle_type == "Car" else "_Bicycling"
        col_name = "Duration_Intervenant_" + str(inter_id) + vehicle_str
        inter_idx = inter_id_to_idx[inter_id]

        for client_id in full_client_list:
            client_idx = client_id_to_idx[client_id]
            try:
                travel_time = convert_duration(
                    df.loc[df["ID Client"] == client_id, col_name].values[0]
                )
                matrix[inter_idx][client_idx] = travel_time
            except IndexError:
                matrix[inter_idx][client_idx] = out_value

    return np.array(matrix)

In [11]:
inter_time_matrix_car = define_inter_time_matrix(
    c_n_dist, full_client_list, vehicle_type="Car"
)
inter_time_matrix_bike = define_inter_time_matrix(
    c_n_dist, full_client_list, vehicle_type="Bike"
)

In [12]:
def create_full_time_matrix(client_time_matrix, inter_time_matrix, out_value=4000):
    nb_client = client_time_matrix.shape[0]
    nb_inter = inter_time_matrix.shape[0]

    matrix = [
        [out_value for i in range(nb_client + nb_inter)]
        for k in range(nb_client + nb_inter)
    ]

    for i in range(nb_client):
        for j in range(nb_client):
            matrix[i][j] = client_time_matrix[i][j]

    for i in range(nb_inter):
        for j in range(nb_client):
            matrix[i + nb_client][j] = inter_time_matrix[i][j]
            matrix[j][i + nb_client] = matrix[i + nb_client][j]

    return np.array(matrix)

In [13]:
full_time_matrix_car = create_full_time_matrix(
    client_time_matrix_car, inter_time_matrix_car
)
full_time_matrix_bike = create_full_time_matrix(
    client_time_matrix_bike, inter_time_matrix_bike
)

In [14]:
list_skills = c_l_need["Prestation"].unique()

condition = c_l_need["Date"] == "2024-01-01"
c_l_test = c_l_need[condition]
c_l_test.drop(columns=["Unnamed: 0", "Date", "Latitude", "Longitude"], inplace=True)
c_l_test

/var/folders/95/2w4zxm7j571dt7gznpftznw80000gn/T/ipykernel_72380/3814535759.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_l_test.drop(columns=["Unnamed: 0", "Date", "Latitude", "Longitude"], inplace=True)


,ID Client,Prestation,Durée,tranche_horaire
0,559475456,REPAS,30.0,"[7, 9]"
1,559277088,TOILETTE,45.0,"[7, 11]"
2,87852633,TOILETTE,45.0,"[7, 11]"
3,243033408,TOILETTE,30.0,"[7, 11]"
4,814940942,TOILETTE,95.0,"[7, 11]"
...,...,...,...,...
61,714782168,REPAS,60.0,"[17, 19.5]"
62,814940942,TOILETTE,60.0,"[16, 21]"
63,530138806,REPAS,30.0,"[17, 19.5]"
64,803656603,TOILETTE,30.0,"[16, 21]"


In [15]:
def create_real_data_model(
    df_need,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
    list_skills=list_skills,
):
    nb_needs = len(df_need)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["time_matrix_car"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["time_matrix_bike"] = [
        [4000 for i in range(nb_needs + nb_inter)] for k in range(nb_needs + nb_inter)
    ]
    data["service_times"] = [0 for i in range(nb_needs + nb_inter)]
    data["time_windows"] = [(0, 1440) for i in range(nb_needs + nb_inter)]

    data["num_vehicles"] = nb_inter
    data["is_car"] = [
        1 for i in range(nb_inter)
    ]  # For the moment every inter has a car
    data["starts"] = [nb_needs + i for i in range(nb_inter)]
    data["ends"] = [nb_needs + i for i in range(nb_inter)]

    data["vehicle_skills"] = [
        list_skills for i in range(nb_inter)
    ]  # For the moment every inter has all skills possible
    data["node_requirements"] = [None for i in range(nb_needs + nb_inter)]

    # Fill with corresponding values
    for i in range(nb_needs):
        need_id_client = df_need["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_needs):
            other_id_client = df_need["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["time_matrix_car"][i][j] = travel_time_car
            data["time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["time_matrix_car"][i][nb_needs + j] = travel_time_car
            data["time_matrix_car"][nb_needs + j][i] = data["time_matrix_car"][i][
                nb_needs + j
            ]
            data["time_matrix_bike"][i][nb_needs + j] = travel_time_bike
            data["time_matrix_bike"][nb_needs + j][i] = data["time_matrix_bike"][i][
                nb_needs + j
            ]

        service_time = int(df_need["Durée"][i])
        data["service_times"][i] = service_time

        time_window_temp = (
            df_need["tranche_horaire"][i]
            .replace("[", "")
            .replace("]", "")
            .replace(",", "")
            .split()
        )
        time_window_start = int(float(time_window_temp[0]) * 60)
        time_window_end = int(float(time_window_temp[1]) * 60)
        data["time_windows"][i] = (time_window_start, time_window_end)

        skills_required = df_need["Prestation"][i]
        data["node_requirements"][i] = skills_required

    for i in range(nb_needs + nb_inter):
        data["time_matrix_car"][i][i] = 0
        data["time_matrix_bike"][i][i] = 0

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    return data

In [16]:
data_test = create_real_data_model(
    c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
)

In [17]:
data_test

{'time_matrix_car': [[0,
   1,
   17,
   9,
   5,
   25,
   3,
   9,
   25,
   14,
   4,
   23,
   4,
   9,
   4,
   22,
   7,
   6,
   2,
   11,
   21,
   9,
   4,
   13,
   7,
   1,
   7,
   25,
   4,
   10,
   8,
   8,
   18,
   9,
   1,
   5,
   0,
   11,
   9,
   23,
   4,
   22,
   8,
   4,
   9,
   17,
   7,
   23,
   9,
   4,
   9,
   11,
   23,
   25,
   13,
   17,
   3,
   1,
   1,
   22,
   0,
   7,
   5,
   8,
   2,
   4,
   4000,
   4,
   10,
   16,
   23,
   4000,
   5,
   4000,
   4,
   4000,
   5,
   16,
   6,
   6,
   18,
   13,
   9,
   1,
   27,
   4000,
   20,
   3,
   7,
   17],
  [1,
   0,
   17,
   9,
   5,
   25,
   3,
   9,
   25,
   14,
   4,
   23,
   4,
   9,
   4,
   22,
   7,
   6,
   2,
   11,
   21,
   9,
   4,
   13,
   7,
   1,
   7,
   25,
   4,
   10,
   8,
   8,
   18,
   9,
   0,
   5,
   1,
   11,
   9,
   23,
   4,
   22,
   8,
   4,
   9,
   17,
   7,
   23,
   9,
   4,
   9,
   11,
   23,
   25,
   13,
   17,
   3,
   1,
   0,
   22,
   1,
   7

In [18]:
def print_detailed_solution_real_data(
    data, manager, routing, solution, inter_idx_to_id=inter_idx_to_id
):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        # vehicle_skills = str(data["vehicle_skills"][vehicle_id])
        inter_id = inter_idx_to_id[vehicle_id]
        # print(f"Route for vehicle {inter_id} (Skills: {vehicle_skills}):")
        print(f"Route for vehicle {inter_id}:")
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            service_time = data["service_times"][manager.IndexToNode(index)]
            skills_required = data["node_requirements"][manager.IndexToNode(index)]
            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                if data["is_car"][vehicle_id]:
                    next_travel_time = data["time_matrix_car"][
                        manager.IndexToNode(index)
                    ][next_node]
                else:
                    next_travel_time = data["time_matrix_bike"][
                        manager.IndexToNode(index)
                    ][next_node]
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )

            break_time = departure_time - end_of_service_time

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            # Print detailed timing for current location
            print(
                f"Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time}, "
                f"End of Service Time (Skills required: {skills_required}): {end_of_service_time}, Break: {break_time}, "
                f"Departure: {departure_time}) -> "
            )

            index = next_index

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        print(
            f"Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time})"
        )
        total_time += final_arrival_time
        print("\n")
    print(f"Total time of all routes: {total_time}min")

In [19]:
def main_real_data(
    c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
):
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_real_data_model(
        c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
    )

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  # Car
            return data["time_matrix_car"][from_node][to_node] + service_time
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)
    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)

    # Enforce node requirements based on vehicle skills
    for node_idx, requirement in enumerate(data["node_requirements"]):
        if requirement:  # Check if the node has specific skill requirements
            allowed_vehicles = [
                vehicle_id
                for vehicle_id, skills in enumerate(data["vehicle_skills"])
                if requirement in skills
            ]
            manager_idx = manager.NodeToIndex(node_idx)
            if manager_idx >= 0:  # Ensure manager index is valid
                routing.SetAllowedVehiclesForIndex(allowed_vehicles, manager_idx)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [bike_index, car_index],
        100,  # no slack
        [300, 300],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        print_detailed_solution_real_data(data, manager, routing, solution)

In [ ]:
main_real_data(c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list)

: 